In [11]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

The yamlmagic extension is already loaded. To reload it, use:
  %reload_ext yamlmagic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [13]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


In [15]:
%%yaml dataset_cfg 

dataset:
    _class: ese.experiment.datasets.WMH
    annotator: observer_o12
    axis: 0
    split: '?'
    slicing: full 
    task: '?'
    version: 0.2

<IPython.core.display.Javascript object>

In [16]:
dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
    'dataset.task': ['Singapore', 'Utrecht', 'Amsterdam']
}

In [17]:
from ionpy.util import dict_product, Config

import copy# Assemble base config
base_cfg = Config(dataset_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    cfg = base_cfg.update(cfg_update)
    cfgs.append(cfg)

In [18]:
len(cfgs)

9

In [19]:
cfgs[0]

Config({'dataset': {'_class': 'ese.experiment.datasets.WMH', 'annotator': 'observer_o12', 'axis': 0, 'split': 'train', 'slicing': 'full', 'task': 'Singapore', 'version': 0.2}})

In [20]:
from ese.experiment.analysis.inference import get_dice_breakdown

get_dice_breakdown(best_exp, cfgs)

Processing Singapore train


100%|██████████| 29/29 [02:35<00:00,  5.35s/it]


Processing Utrecht train


100%|██████████| 29/29 [04:07<00:00,  8.54s/it]


Processing Amsterdam train


...

Processing Utrecht cal


100%|██████████| 5/5 [00:45<00:00,  9.07s/it]


Processing Amsterdam cal


100%|██████████| 7/7 [00:57<00:00,  8.15s/it]


ArrowInvalid: ("Could not convert <class 'slice'> with type type: did not recognize Python value type when inferring an Arrow data type", 'Conversion failed for column slice with type object')